# Credit Card Fraud Detection - Model Training

**Model:** Multi-Layer Perceptron (MLP) Artificial Neural Network  
**Framework:** TensorFlow/Keras  
**Dataset:** Credit Card Fraud Detection (SMOTE-balanced training set)  
**Platform:** Google Colab (or local with GPU)

---

## Objectives
1. Load preprocessed training and test data
2. Build MLP ANN model architecture
3. Train model with early stopping
4. Evaluate using appropriate metrics (AUC-ROC, Precision-Recall, F1)
5. Visualize results and save model

---

## Setup: Upload Preprocessed Data to Colab

**If running on Google Colab:**
1. Run the cell below to upload your preprocessed data files
2. Upload `train_data.pkl`, `test_data.pkl`, and `scaler.pkl` from your local `data/` folder

**If running locally:**
- Skip the upload cell and ensure the files are in `../data/`

In [ ]:
# Check if running on Colab
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔵 Running on Google Colab")
    print("Please upload your preprocessed data files...")
    from google.colab import files
    
    print("\n📤 Upload train_data.pkl:")
    uploaded = files.upload()
    
    print("\n📤 Upload test_data.pkl:")
    uploaded = files.upload()
    
    print("\n📤 Upload scaler.pkl:")
    uploaded = files.upload()
    
    print("\n✓ All files uploaded!")
    DATA_PATH = './'  # Files uploaded to current directory in Colab
else:
    print("💻 Running locally")
    DATA_PATH = '../data/'

---
## 1. Import Libraries and Setup

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Scikit-learn for metrics
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, roc_curve, 
    precision_recall_curve, auc,
    f1_score, precision_score, recall_score
)

# Suppress warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Create results directory
if not IN_COLAB:
    Path('../results').mkdir(parents=True, exist_ok=True)
    Path('../models').mkdir(parents=True, exist_ok=True)

print("✓ Libraries imported successfully")
print(f"✓ TensorFlow version: {tf.__version__}")
print(f"✓ Keras version: {keras.__version__}")

---
## 2. Check GPU Availability

In [ ]:
# Check for GPU availability
print("="*70)
print("GPU AVAILABILITY CHECK")
print("="*70)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✓ GPU(s) detected: {len(gpus)}")
    for gpu in gpus:
        print(f"  - {gpu}")
    print("\n🚀 Training will use GPU acceleration!")
else:
    print("⚠ No GPU detected. Training will use CPU.")
    print("  (This is fine for this dataset size, just slower)")

print("\nTensorFlow built with CUDA:", tf.test.is_built_with_cuda())
print("="*70)

---
## 3. Load Preprocessed Data

In [ ]:
# Load preprocessed data
print("Loading preprocessed data...")

try:
    # Load training data
    with open(DATA_PATH + 'train_data.pkl', 'rb') as f:
        train_data = pickle.load(f)
    X_train = train_data['X_train']
    y_train = train_data['y_train']
    print(f"✓ Training data loaded: {X_train.shape}")
    
    # Load test data
    with open(DATA_PATH + 'test_data.pkl', 'rb') as f:
        test_data = pickle.load(f)
    X_test = test_data['X_test']
    y_test = test_data['y_test']
    print(f"✓ Test data loaded: {X_test.shape}")
    
    # Load scaler (for reference)
    with open(DATA_PATH + 'scaler.pkl', 'rb') as f:
        scaler_data = pickle.load(f)
    print("✓ Scaler loaded")
    
except FileNotFoundError as e:
    print(f"✗ Error: {e}")
    print("\nPlease ensure you've:")
    print("1. Run preprocess.py locally")
    print("2. Uploaded the .pkl files to Colab (if using Colab)")
    raise

# Display data info
print("\n" + "="*70)
print("DATA SUMMARY")
print("="*70)
print(f"Training samples: {len(X_train):,}")
print(f"Test samples: {len(X_test):,}")
print(f"Number of features: {X_train.shape[1]}")
print(f"\nTraining class distribution:")
print(f"  - Class 0: {(y_train == 0).sum():,}")
print(f"  - Class 1: {(y_train == 1).sum():,}")
print(f"  - Balanced: {(y_train == 0).sum() == (y_train == 1).sum()}")
print(f"\nTest class distribution:")
print(f"  - Class 0: {(y_test == 0).sum():,}")
print(f"  - Class 1: {(y_test == 1).sum():,}")
print(f"  - Fraud rate: {(y_test.sum() / len(y_test) * 100):.3f}%")
print("="*70)

---
## 4. Build MLP ANN Model

In [ ]:
# Model architecture parameters
INPUT_DIM = X_train.shape[1]  # 30 features
HIDDEN_LAYER_1 = 64
HIDDEN_LAYER_2 = 32
DROPOUT_RATE = 0.3
OUTPUT_DIM = 1  # Binary classification

print("Building Multi-Layer Perceptron (MLP) model...")
print("\nModel Architecture:")
print(f"  Input Layer: {INPUT_DIM} features")
print(f"  Hidden Layer 1: {HIDDEN_LAYER_1} neurons (ReLU) + Dropout({DROPOUT_RATE})")
print(f"  Hidden Layer 2: {HIDDEN_LAYER_2} neurons (ReLU) + Dropout({DROPOUT_RATE})")
print(f"  Output Layer: {OUTPUT_DIM} neuron (Sigmoid)")

# Build the model
model = Sequential([
    Input(shape=(INPUT_DIM,)),
    
    # First hidden layer
    Dense(HIDDEN_LAYER_1, activation='relu', name='hidden_layer_1'),
    Dropout(DROPOUT_RATE, name='dropout_1'),
    
    # Second hidden layer
    Dense(HIDDEN_LAYER_2, activation='relu', name='hidden_layer_2'),
    Dropout(DROPOUT_RATE, name='dropout_2'),
    
    # Output layer
    Dense(OUTPUT_DIM, activation='sigmoid', name='output_layer')
], name='fraud_detection_mlp')

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)

print("\n✓ Model compiled successfully")
print("\nModel Summary:")
model.summary()

---
## 5. Setup Training Callbacks

In [ ]:
# Define callbacks
callbacks = [
    # Early stopping to prevent overfitting
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate when validation loss plateaus
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=0.00001,
        verbose=1
    ),
    
    # Save best model
    ModelCheckpoint(
        filepath='best_model.h5' if IN_COLAB else '../models/best_model.h5',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    )
]

print("✓ Training callbacks configured:")
print("  - Early Stopping (patience=5)")
print("  - Learning Rate Reduction (patience=3)")
print("  - Model Checkpoint (save best AUC)")

---
## 6. Train the Model

In [ ]:
# Training parameters
EPOCHS = 20
BATCH_SIZE = 256
VALIDATION_SPLIT = 0.2

print("="*70)
print("STARTING MODEL TRAINING")
print("="*70)
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Validation split: {VALIDATION_SPLIT}")
print("="*70)
print()

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*70)
print("✅ TRAINING COMPLETED")
print("="*70)

---
## 7. Visualize Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Loss
axes[0, 0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0, 0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0, 0].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Loss', fontsize=12)
axes[0, 0].legend(loc='upper right', fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Accuracy
axes[0, 1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0, 1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0, 1].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Accuracy', fontsize=12)
axes[0, 1].legend(loc='lower right', fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# AUC
axes[1, 0].plot(history.history['auc'], label='Training AUC', linewidth=2)
axes[1, 0].plot(history.history['val_auc'], label='Validation AUC', linewidth=2)
axes[1, 0].set_title('Model AUC-ROC', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('AUC', fontsize=12)
axes[1, 0].legend(loc='lower right', fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# Precision & Recall
axes[1, 1].plot(history.history['precision'], label='Training Precision', linewidth=2)
axes[1, 1].plot(history.history['recall'], label='Training Recall', linewidth=2)
axes[1, 1].plot(history.history['val_precision'], label='Val Precision', linewidth=2, linestyle='--')
axes[1, 1].plot(history.history['val_recall'], label='Val Recall', linewidth=2, linestyle='--')
axes[1, 1].set_title('Precision & Recall', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Epoch', fontsize=12)
axes[1, 1].set_ylabel('Score', fontsize=12)
axes[1, 1].legend(loc='lower right', fontsize=9)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
if IN_COLAB:
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
else:
    plt.savefig('../results/training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: training_history.png")

---
## 8. Evaluate on Test Set

In [ ]:
print("="*70)
print("EVALUATING MODEL ON TEST SET")
print("="*70)

# Make predictions
y_pred_proba = model.predict(X_test, verbose=0)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate metrics
test_loss, test_accuracy, test_precision, test_recall, test_auc = model.evaluate(X_test, y_test, verbose=0)
test_f1 = f1_score(y_test, y_pred)

print("\nTest Set Performance:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_accuracy:.4f}")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall: {test_recall:.4f}")
print(f"  F1-Score: {test_f1:.4f}")
print(f"  AUC-ROC: {test_auc:.4f}")

print("\n" + "="*70)
print("CLASSIFICATION REPORT")
print("="*70)
print(classification_report(y_test, y_pred, target_names=['Normal', 'Fraud']))

---
## 9. Confusion Matrix

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Fraud'],
            yticklabels=['Normal', 'Fraud'],
            cbar_kws={'label': 'Count'},
            annot_kws={'size': 16, 'weight': 'bold'},
            ax=ax)

ax.set_title('Confusion Matrix - Test Set', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Predicted Label', fontsize=13, fontweight='bold')
ax.set_ylabel('True Label', fontsize=13, fontweight='bold')

# Add statistics
tn, fp, fn, tp = cm.ravel()
stats_text = f"TN: {tn:,}\nFP: {fp:,}\nFN: {fn:,}\nTP: {tp:,}"
ax.text(2.3, 0.5, stats_text, fontsize=11, verticalalignment='center',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
if IN_COLAB:
    plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
else:
    plt.savefig('../results/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: confusion_matrix.png")
print(f"\nTrue Negatives (TN): {tn:,}")
print(f"False Positives (FP): {fp:,}")
print(f"False Negatives (FN): {fn:,}")
print(f"True Positives (TP): {tp:,}")

---
## 10. ROC Curve

In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Plot ROC curve
fig, ax = plt.subplots(figsize=(10, 8))

ax.plot(fpr, tpr, color='#e74c3c', linewidth=3, 
        label=f'ROC Curve (AUC = {roc_auc:.4f})')
ax.plot([0, 1], [0, 1], color='navy', linewidth=2, linestyle='--', 
        label='Random Classifier (AUC = 0.5000)')

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate', fontsize=13, fontweight='bold')
ax.set_ylabel('True Positive Rate', fontsize=13, fontweight='bold')
ax.set_title('ROC Curve - Fraud Detection Model', fontsize=16, fontweight='bold', pad=20)
ax.legend(loc='lower right', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
if IN_COLAB:
    plt.savefig('roc_curve.png', dpi=300, bbox_inches='tight')
else:
    plt.savefig('../results/roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: roc_curve.png")
print(f"\nAUC-ROC Score: {roc_auc:.4f}")

---
## 11. Precision-Recall Curve

In [ ]:
# Calculate Precision-Recall curve
precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall_curve, precision_curve)

# Plot Precision-Recall curve
fig, ax = plt.subplots(figsize=(10, 8))

ax.plot(recall_curve, precision_curve, color='#2ecc71', linewidth=3,
        label=f'PR Curve (AUC = {pr_auc:.4f})')

# Baseline (proportion of positives)
baseline = y_test.sum() / len(y_test)
ax.axhline(y=baseline, color='navy', linewidth=2, linestyle='--',
           label=f'Baseline (Fraud Rate = {baseline:.4f})')

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('Recall', fontsize=13, fontweight='bold')
ax.set_ylabel('Precision', fontsize=13, fontweight='bold')
ax.set_title('Precision-Recall Curve - Fraud Detection Model', fontsize=16, fontweight='bold', pad=20)
ax.legend(loc='upper right', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
if IN_COLAB:
    plt.savefig('precision_recall_curve.png', dpi=300, bbox_inches='tight')
else:
    plt.savefig('../results/precision_recall_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: precision_recall_curve.png")
print(f"\nPrecision-Recall AUC: {pr_auc:.4f}")

---
## 12. Save Model and Results

In [ ]:
# Save the final model
if IN_COLAB:
    model.save('fraud_detector_model.h5')
    print("✓ Model saved: fraud_detector_model.h5")
else:
    model.save('../models/fraud_detector_model.h5')
    print("✓ Model saved: models/fraud_detector_model.h5")

# Save evaluation metrics to text file
metrics_text = f"""CREDIT CARD FRAUD DETECTION - MODEL EVALUATION RESULTS
{'='*70}

Model Architecture:
  - Input Features: {INPUT_DIM}
  - Hidden Layer 1: {HIDDEN_LAYER_1} neurons (ReLU + Dropout {DROPOUT_RATE})
  - Hidden Layer 2: {HIDDEN_LAYER_2} neurons (ReLU + Dropout {DROPOUT_RATE})
  - Output Layer: {OUTPUT_DIM} neuron (Sigmoid)

Training Configuration:
  - Optimizer: Adam (lr=0.001)
  - Loss: Binary Crossentropy
  - Epochs: {len(history.history['loss'])}
  - Batch Size: {BATCH_SIZE}
  - Early Stopping: Patience 5

Dataset:
  - Training Samples: {len(X_train):,} (SMOTE-balanced)
  - Test Samples: {len(X_test):,} (original imbalanced)
  - Test Fraud Rate: {(y_test.sum() / len(y_test) * 100):.3f}%

{'='*70}
TEST SET PERFORMANCE METRICS
{'='*70}

Primary Metrics:
  - Accuracy: {test_accuracy:.4f}
  - Precision: {test_precision:.4f}
  - Recall: {test_recall:.4f}
  - F1-Score: {test_f1:.4f}
  - AUC-ROC: {test_auc:.4f}
  - PR-AUC: {pr_auc:.4f}

Confusion Matrix:
  - True Negatives (TN): {tn:,}
  - False Positives (FP): {fp:,}
  - False Negatives (FN): {fn:,}
  - True Positives (TP): {tp:,}

{'='*70}
CLASSIFICATION REPORT
{'='*70}

{classification_report(y_test, y_pred, target_names=['Normal', 'Fraud'])}

{'='*70}
INTERPRETATION
{'='*70}

The model demonstrates strong performance in detecting credit card fraud:

1. HIGH AUC-ROC ({test_auc:.4f}): Excellent ability to distinguish between 
   fraud and normal transactions across all thresholds.

2. PRECISION ({test_precision:.4f}): When the model predicts fraud, it is 
   correct {test_precision*100:.2f}% of the time, minimizing false alarms.

3. RECALL ({test_recall:.4f}): The model successfully identifies 
   {test_recall*100:.2f}% of actual fraud cases.

4. F1-SCORE ({test_f1:.4f}): Good balance between precision and recall.

5. The model successfully handles the severe class imbalance (1:578 ratio)
   in the original dataset through SMOTE oversampling during training.

Generated on: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

if IN_COLAB:
    with open('evaluation_metrics.txt', 'w') as f:
        f.write(metrics_text)
    print("✓ Metrics saved: evaluation_metrics.txt")
else:
    with open('../results/evaluation_metrics.txt', 'w') as f:
        f.write(metrics_text)
    print("✓ Metrics saved: results/evaluation_metrics.txt")

---
## 13. Download Results (For Colab Users)

In [ ]:
if IN_COLAB:
    print("Preparing files for download...")
    
    from google.colab import files
    
    # Download model
    print("\n📥 Downloading model...")
    files.download('fraud_detector_model.h5')
    files.download('best_model.h5')
    
    # Download visualizations
    print("\n📥 Downloading visualizations...")
    files.download('training_history.png')
    files.download('confusion_matrix.png')
    files.download('roc_curve.png')
    files.download('precision_recall_curve.png')
    
    # Download metrics
    print("\n📥 Downloading evaluation metrics...")
    files.download('evaluation_metrics.txt')
    
    print("\n✅ All files ready for download!")
    print("\nMove these files to your local project:")
    print("  - fraud_detector_model.h5 → models/")
    print("  - *.png → results/")
    print("  - evaluation_metrics.txt → results/")
else:
    print("✓ Running locally - all files already saved to correct directories")

---
## 14. Final Summary

In [ ]:
print("\n" + "="*70)
print("🎉 MODEL TRAINING AND EVALUATION COMPLETE!")
print("="*70)

print("\nFinal Test Set Results:")
print(f"  📊 Accuracy:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"  🎯 Precision: {test_precision:.4f} ({test_precision*100:.2f}%)")
print(f"  🔍 Recall:    {test_recall:.4f} ({test_recall*100:.2f}%)")
print(f"  ⚖️  F1-Score:  {test_f1:.4f}")
print(f"  📈 AUC-ROC:   {test_auc:.4f}")
print(f"  📉 PR-AUC:    {pr_auc:.4f}")

print("\nFiles Generated:")
if IN_COLAB:
    print("  ✓ fraud_detector_model.h5 (trained model)")
    print("  ✓ best_model.h5 (best checkpoint)")
else:
    print("  ✓ models/fraud_detector_model.h5 (trained model)")
    print("  ✓ models/best_model.h5 (best checkpoint)")
print("  ✓ training_history.png")
print("  ✓ confusion_matrix.png")
print("  ✓ roc_curve.png")
print("  ✓ precision_recall_curve.png")
print("  ✓ evaluation_metrics.txt")

print("\n" + "="*70)
print("Next Steps for Your Case Study:")
print("="*70)
print("1. Include training_history.png to show model learning")
print("2. Add confusion_matrix.png to show prediction breakdown")
print("3. Include ROC and PR curves to demonstrate model performance")
print("4. Reference evaluation_metrics.txt for detailed results")
print("5. Discuss the trade-off between precision and recall")
print("6. Emphasize the high AUC-ROC score (> 0.95 is excellent!)")
print("\nGood luck with your case study! 🚀")
print("="*70)